<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [48]:
%%writefile get_data.py
"""
Script that download data and unpack it in data folder.
"""
# YOUR CODE HERE
import os
from pathlib import Path
import requests
import zipfile
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

BATCH_SIZE = 1
SIMPLE_TRANSFORM = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

DATA_PATH = Path("data/")
IMAGE_PATH = DATA_PATH / 'pizza_steak_sushi'

TRAIN_PATH = IMAGE_PATH / 'train'
TEST_PATH = IMAGE_PATH / 'test'

if IMAGE_PATH.is_dir():
  print(f'Directory: {IMAGE_PATH} already exists.')
else:
  IMAGE_PATH.mkdir(parents=True, exist_ok=True)

  with open(DATA_PATH / "pizza_steak_sushi.zip", 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    f.write(request.content)

  with zipfile.ZipFile(DATA_PATH / "pizza_steak_sushi.zip", 'r') as zipf:
    zipf.extractall(IMAGE_PATH)

  os.remove(DATA_PATH / 'pizza_steak_sushi.zip')

def prepare_data(train_path: Path,
                 test_path: Path,
                 transform: torchvision.transforms) -> dict():
  """
  Function to prepare data (train/test data, train/test dataloaders and class_names) and return it as a dict
  Returns dict
  """

  prepared_data = {'train_data': None,
                   'test_data': None,
                   'train_dataloader': None,
                   'test_dataloader': None,
                   'class_names': None}

  # Train data
  train_data = ImageFolder(train_path,
                          transform=transform,
                          target_transform=None)

  test_data = ImageFolder(test_path,
                          transform = transform,
                          target_transform=None)

  # Dataloaders
  train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True)

  test_dataloader = DataLoader(test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

  # Class names
  class_names = test_data.classes

  prepared_data['train_data'], prepared_data['test_data'] = train_data, test_data
  prepared_data['train_dataloader'], prepared_data['test_dataloader'] = train_dataloader, test_dataloader
  prepared_data['class_names'] = class_names

  return prepared_data

Directory: data/pizza_steak_sushi already exists.


In [2]:
# Example running of get_data.py
!python get_data.py

## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [26]:
%%writefile model.py

from torch import nn
import torch

class TinyVGG(nn.Module):
  def __init__(self,
               input_channels:int,
               hidden_channels:int,
               output_channels:int):
    super().__init__()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2,
                     padding=0)
    )
    self.layer_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2,
                     padding=0)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_channels * 13 * 13,
                  out_features = output_channels)
    )
  def forward(self, x):
    x = self.layer_1(x)
    print(x.shape)
    x = self.layer_2(x)
    print(x.shape)
    x = self.classifier(x)
    return x

Writing model.py


In [27]:
%%writefile utils.py

from torch.utils.data import DataLoader
import torch
from torch import nn
from typing import Tuple

def train_step(dataloader: DataLoader,
               model:nn.Module,
               loss_fn:nn.Module,
               optimizer:torch.optim,
               device:torch.device
               ) -> Tuple[float, float]:
  """
  Training step for a model as a function.
  Returns train_loss and train step
  """
  model.train()

  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    # Forward pass
    y_logit = model(X)

    # Calculate the loss
    loss = loss_fn(y_logit, y)
    train_loss += loss.item()

    # Optimizer zero grad
    optimizer.zero_grad()

    # Loss backward
    loss.backward()

    # Optimizer step step
    optimizer.step()

    # Accuracy
    y_pred_labels = torch.argmax(torch.softmax(y_logit, dim=1), dim=1)
    train_acc += ((y_pred_labels == y).sum().item()/len(y))
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

def test_step(dataloader: DataLoader,
              model: nn.Module,
              loss_fn: nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """
  Testing step for model in a function
  Returns test_loss and test_acc
  """
  test_loss, test_acc = 0, 0

  model.eval()

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
    # Forward pass
      y_logit = model(X)

    # Calculate the loss
      loss = loss_fn(y_logit, y)
      test_loss += loss

    # Calculate acc
      test_pred_labels = torch.argmax(torch.softmax(y_logit, dim=1), dim=1)
      test_acc = ((test_pred_labels == y).sum().item() / len(y))

  test_loss /= len(dataloader)
  test_acc /= len(dataloader)

  return test_loss, test_acc

Overwriting utils.py


In [30]:
# YOUR CODE HERE
import torch
from torch import nn
import os
from tqdm.auto import tqdm
from timeit import default_timer as timer

import model
import utils

def train(train_dir: os.path,
          test_dir: os.path,
          model: nn.Module,
          optimizer: torch.optim,
          lr:int,
          loss_fn:nn.Module,

          batch_size:int,
          epochs:int):
  start_train = timer()

  for epoch in tqdm(range(epochs)):
    utils.test_step()


In [ ]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [ ]:
# YOUR CODE HERE

In [ ]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg